# Lesson 11 - Spark Execution Model

Okay, let's craft the technical notes for Lesson 11, focusing on the Spark Execution Model, DAG Scheduler, Catalyst Optimizer, and Tungsten.

---

## PySpark Technical Notes: Lesson 11 - Understanding Spark's Execution Engine

### Introduction

To write efficient and scalable PySpark applications, it's crucial to look beyond the APIs and understand how Spark translates your code into distributed computations. This lesson delves into the core components of Spark's execution engine: the hierarchy of Jobs, Stages, and Tasks; the role of the DAG Scheduler in planning execution; the sophisticated Catalyst Optimizer that refines DataFrame and SQL queries; and the Tungsten execution engine that maximizes hardware efficiency. Mastering these concepts is key to performance tuning and debugging complex Spark applications.

### The Spark Execution Hierarchy: Jobs, Stages, and Tasks

**Theory:**

When you submit a Spark application, the Driver program orchestrates the execution across the cluster's Executor nodes. Spark breaks down the overall computation defined by your code into a hierarchy:

1.  **Application:** The highest level, representing your entire program submitted to Spark (e.g., your `spark-submit` script or interactive session). Managed by the Driver process.
2.  **Job:** A parallel computation triggered by a Spark **Action**. Actions are operations that return a result to the Driver (e.g., `count()`, `collect()`, `save()`) or write data to an external storage system. One Spark application can consist of multiple jobs, executed sequentially or concurrently depending on the code logic.
3.  **Stage:** Each Job is divided into one or more Stages. Stages represent groups of tasks that can be executed together *without* a data shuffle (redistribution of data across partitions). The boundaries between stages are determined by **shuffle dependencies** (wide dependencies), which occur during operations like `groupByKey`, `reduceByKey`, `join`, `repartition`, etc., where data needs to be moved across the network between executors.
4.  **Task:** The smallest unit of execution. Each Stage consists of multiple Tasks, where each Task operates on a single **partition** of the data and executes the same computation logic. Tasks are launched by the Driver on available Executor slots. The number of tasks in a stage typically corresponds to the number of partitions of the RDD/DataFrame being processed at that stage's start.

**Visualizing the Flow:**

```
+-----------------------+
|   Spark Application   | (Your Script/Session)
+----------+------------+
           |
           v
+-----------------------+      +-----------------------+
|         Job 1         | ---> |         Job 2         | (Triggered by Action 1) (Triggered by Action 2)
+----------+------------+      +----------+------------+
           |                         |
           v                         v
+-----------------------+      +-----------------------+
|       Stage 1.1       | ---> |       Stage 1.2       | (Shuffle Boundary)
+----------+------------+      +----------+------------+
           |                         |
           v                         v
[Task 1] [Task 2] ...       [Task 1] [Task 2] ...
(Operate on Partition 1, 2...) (Operate on Shuffled Partitions)
```

**Code Example:**

Let's illustrate this with a simple PySpark sequence.

```python
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("ExecutionModelExample") \
    .config("spark.sql.shuffle.partitions", "4") # Control parallelism for shuffle
    .getOrCreate()

# Create a DataFrame
data = [("Alice", "HR", 50000),
        ("Bob", "Engineering", 80000),
        ("Charlie", "Engineering", 90000),
        ("David", "HR", 60000),
        ("Eve", "Finance", 75000)]
columns = ["name", "department", "salary"]
df = spark.createDataFrame(data, columns).repartition(2) # Initial partitions = 2

print(f"Initial number of partitions: {df.rdd.getNumPartitions()}")

# Transformation 1: Filter (Narrow Dependency)
filtered_df = df.filter(F.col("salary") > 60000)

# Transformation 2: Add a column (Narrow Dependency)
enriched_df = filtered_df.withColumn("bonus", F.col("salary") * 0.1)

# Transformation 3: Group by department (Wide Dependency - Shuffle!)
agg_df = enriched_df.groupBy("department").agg(F.avg("salary").alias("avg_salary"))

# Action 1: Collect the results (Triggers Job 1)
print("\n--- Triggering Job 1 (collect) ---")
results = agg_df.collect()
print("Aggregation Results:")
for row in results:
    print(row)

# Action 2: Count the original filtered data (Triggers Job 2)
# Note: Spark might optimize and reuse some results if possible,
# but conceptually, this triggers separate work based on 'filtered_df'.
print("\n--- Triggering Job 2 (count) ---")
count_filtered = filtered_df.count()
print(f"\nCount of employees with salary > 60000: {count_filtered}")

# Stop the SparkSession
spark.stop()
```

**Code Explanation & Execution Analysis:**

1.  `spark = SparkSession.builder...getOrCreate()`: Initializes the Spark Application.
2.  `df = spark.createDataFrame(...).repartition(2)`: Creates a DataFrame with initially 2 partitions.
3.  `filtered_df = df.filter(...)`: This is a **narrow transformation**. Each output partition depends only on a single input partition. No shuffle needed.
4.  `enriched_df = filtered_df.withColumn(...)`: Another **narrow transformation**. Calculations happen independently within each partition.
5.  `agg_df = enriched_df.groupBy("department").agg(...)`: This is a **wide transformation**. To calculate the average salary per department, Spark needs to gather all rows for the same department onto the same executor/partition. This requires a **shuffle** of the data across the network. `spark.sql.shuffle.partitions` (set to 4 here) determines the number of partitions *after* the shuffle.
6.  `results = agg_df.collect()`: This is the first **Action**. It triggers **Job 1**.
    *   **Job 1** execution:
        *   Spark analyzes the lineage (`agg_df` depends on `enriched_df`, which depends on `filtered_df`, etc.).
        *   It identifies the shuffle required by `groupBy`. This creates a **Stage boundary**.
        *   **Stage 1:** Executes the `filter` and `withColumn` transformations. Since the input `df` had 2 partitions, this stage will likely have 2 Tasks. The output of this stage is intermediate shuffled data (Map stage in MapReduce terms).
        *   **Stage 2:** Reads the shuffled data (now partitioned by `department`, likely into 4 partitions as configured) and performs the `groupBy` and `agg` operations. This stage will have 4 Tasks (matching `spark.sql.shuffle.partitions`). The output is the final `agg_df`.
        *   The `collect()` action gathers the results from the final partitions (output of Stage 2) back to the Driver.
7.  `count_filtered = filtered_df.count()`: This is the second **Action**. It triggers **Job 2**.
    *   **Job 2** execution:
        *   Spark analyzes the lineage back to `filtered_df`.
        *   This calculation only involves narrow transformations (`filter`) from the original `df`. No shuffle is needed for this specific action.
        *   **Stage 3:** Executes the `filter` operation on the initial 2 partitions and counts the rows within each partition. This stage will have 2 Tasks.
        *   The `count()` action aggregates the partial counts from each task at the Driver.

**Practical Use Case:** Understanding this hierarchy is crucial for debugging performance issues using the Spark UI. The UI displays Applications, Jobs, Stages, and Tasks, allowing you to pinpoint slow stages (often due to shuffles or data skew) or failing tasks.

### DAG (Directed Acyclic Graph) Scheduler

**Theory:**

The DAG Scheduler is a key component within the Spark Driver. Its primary responsibility is to take the logical execution plan (represented by the RDD lineage or the optimized logical plan from Catalyst) and convert it into a physical execution plan consisting of a Directed Acyclic Graph (DAG) of Stages.

**How it Works:**

1.  **Receives Logical Plan:** When an Action is called, the DAG Scheduler receives the final RDD or DataFrame in the lineage.
2.  **Computes Lineage Graph:** It traces the dependencies backwards from the final RDD/DataFrame through all transformations.
3.  **Identifies Shuffle Boundaries:** It examines the dependencies between RDDs/DataFrames.
    *   **Narrow Dependencies:** Where each partition of the parent RDD/DataFrame is used by at most one partition of the child RDD/DataFrame (e.g., `map`, `filter`). These operations can be pipelined within a single stage.
    *   **Wide Dependencies (Shuffle Dependencies):** Where partitions of the child RDD/DataFrame depend on multiple parent partitions (e.g., `groupByKey`, `join` where keys are not co-partitioned). Wide dependencies require data shuffling across the network and mark the boundary between Stages.
4.  **Creates Stages:** The DAG Scheduler groups consecutive transformations with narrow dependencies into single Stages. Each wide dependency results in a new Stage.
5.  **Generates DAG of Stages:** The result is a DAG where Stages are nodes, and edges represent dependencies (data flowing from one stage to the next, often involving a shuffle). "Acyclic" means the graph has no loops – execution flows forward.
6.  **Submits Stages:** The DAG Scheduler determines the order of stage execution based on the DAG. It submits ready stages (those whose parent stages have completed) to the Task Scheduler for execution on the cluster. It keeps track of which partitions have been computed (cached or stored as shuffle output) to handle failures and avoid recomputation.

**Fault Tolerance:** The DAG is fundamental to Spark's fault tolerance. If a task fails, Spark knows (from the DAG) which stage it belonged to and which partition it was processing. It can re-launch just that task on another executor. If an executor fails entirely, losing shuffle output, the DAG Scheduler knows which parent stages need to be re-executed to recreate the lost data partitions.

**Example Visualization (Conceptual):**

For the `agg_df.collect()` action above:

```
        +---------------------+        +---------------------+
Input-->| Stage 1 (Tasks: 2)  |------->| Stage 2 (Tasks: 4)  |--> Output (Driver)
        | - df.filter()       | Shuffle| - groupBy().agg()   |
        | - .withColumn()     |        |                     |
        +---------------------+        +---------------------+
        (Narrow Dependencies)         (Reads Shuffled Data)
```

### Catalyst Optimizer

**Theory:**

Catalyst is Spark SQL's extensible query optimizer, forming the core of the DataFrame and SQL APIs. It translates the high-level, declarative operations (SQL queries, DataFrame transformations) into efficient physical execution plans. It leverages advanced programming language features (Scala's pattern matching and quasiquotes) to build a highly modular and extensible optimizer.

**Optimization Phases:**

Catalyst applies a series of rule-based and cost-based optimizations in phases:

1.  **Analysis:**
    *   Resolves relations and attributes: Checks if tables, views, and columns mentioned in the query or DataFrame operations exist in the catalog (metadata).
    *   Assigns types: Determines the data types of expressions.
    *   Builds an **Unresolved Logical Plan**.
    *   Resolves identifiers using the catalog, producing a **Logical Plan**.

2.  **Logical Optimization:**
    *   Applies standard rule-based optimizations to the Logical Plan. Examples include:
        *   **Predicate Pushdown:** Pushing `filter` operations closer to the data source (e.g., into Parquet file reads) to reduce the amount of data scanned.
        *   **Constant Folding:** Evaluating constant expressions at compile time (e.g., `1 + 1` becomes `2`).
        *   **Column Pruning:** Removing unused columns early in the plan to minimize data movement.
        *   **Projection Pushdown:** Similar to predicate pushdown but for selecting specific columns.
        *   Boolean expression simplification, null propagation, etc.
    *   Produces an **Optimized Logical Plan**.

3.  **Physical Planning:**
    *   Generates one or more **Physical Plans** from the Optimized Logical Plan. Physical plans describe *how* the query will be executed (e.g., which join algorithms to use - Broadcast Hash Join, Sort Merge Join).
    *   Takes into account data statistics (if available) and physical properties of operators.
    *   Applies physical optimizations like ensuring correct data distribution for joins or aggregations.

4.  **Cost-Based Optimization (CBO):**
    *   If multiple Physical Plans are generated, Catalyst uses a cost model (based on data statistics like row count, distinct values, null counts) to estimate the execution cost of each plan.
    *   Selects the **Best Physical Plan** (lowest estimated cost). Accurate statistics (gathered via `ANALYZE TABLE COMPUTE STATISTICS`) are crucial for CBO to be effective.

5.  **Code Generation:**
    *   The final phase, often integrated with the Tungsten execution engine.
    *   Translates the selected Physical Plan into highly optimized Java bytecode (Whole-Stage Code Generation) that runs efficiently on the JVM, minimizing virtual function calls and leveraging CPU registers. (See Tungsten section).

**Why Catalyst Matters:**

*   **Performance:** It's the primary reason why DataFrame and SQL APIs generally outperform RDD operations for structured data. It automatically applies complex optimizations that would be difficult or tedious to implement manually with RDDs.
*   **Extensibility:** Allows external data sources and user-defined functions/aggregations to integrate with the optimization process.
*   **Unified Engine:** Optimizes queries regardless of whether they originate from SQL strings, DataFrame API calls, or streaming sources.

**Checking the Plan:** You can inspect the plans generated by Catalyst using `DataFrame.explain()` or `spark.sql("EXPLAIN ...").show(truncate=False)`.

```python
# Assuming agg_df from the previous example
print("\n--- Catalyst Optimizer Plans ---")
agg_df.explain(extended=True) # Show logical and physical plans
```

**Explanation Output Analysis:**

*   `== Parsed Logical Plan ==`: Initial plan before analysis.
*   `== Analyzed Logical Plan ==`: Plan after resolving attributes and relations.
*   `== Optimized Logical Plan ==`: Plan after rule-based logical optimizations (e.g., notice if filters are pushed down).
*   `== Physical Plan ==`: The final plan chosen for execution, showing specific operators (e.g., `HashAggregate`, `Exchange hashpartitioning`, `Filter`, `FileScan`). This reveals shuffles (`Exchange`) and low-level operations.

### Tungsten Execution Engine

**Theory:**

Tungsten is the codename for a broad initiative within Spark focused on maximizing the efficiency of Spark computations by optimizing CPU and memory usage. It operates beneath the SQL/DataFrame layer, improving the performance of the physical operators selected by Catalyst. It aims to approach the performance limits of modern hardware.

**Key Techniques:**

1.  **Whole-Stage Code Generation:** Instead of interpreting each operator in a stage one-by-one for each row, Tungsten compiles entire stages of physical operators into a single, optimized Java bytecode function. This eliminates virtual function call overhead between operators within a stage and allows the JVM's JIT compiler to perform further optimizations like loop unrolling and register allocation, significantly improving CPU efficiency.
2.  **Optimized Memory Management (Unsafe / Off-Heap Memory):**
    *   Tungsten utilizes explicit memory management, often operating directly on binary data in `sun.misc.Unsafe` (or equivalent) memory regions, bypassing JVM object overhead and garbage collection (GC) pressure.
    *   Data is stored in a compact binary format (Tungsten Binary Format) which is efficient for serialization/deserialization and processing.
    *   This allows Spark to handle larger datasets without running into JVM GC limitations and enables more predictable memory usage.
3.  **Cache-Aware Computation:** Algorithms and data structures used by Tungsten (like specialized hash maps for aggregation and joins) are designed to be mindful of CPU cache hierarchies (L1/L2/L3). Keeping frequently accessed data within CPU caches drastically speeds up processing.
4.  **Vectorization (e.g., Parquet/ORC):** When reading columnar formats like Parquet or ORC, Tungsten can leverage vectorized readers that operate on batches of column values at a time, improving CPU pipeline utilization and reducing interpretation overhead compared to row-by-row processing.

**Impact:**

*   **Speed:** Significantly faster execution, especially for CPU-bound workloads involving aggregations, joins, and sorting on structured data.
*   **Memory Efficiency:** Reduced memory footprint per record and less GC overhead, allowing applications to process more data with the same amount of memory.
*   **Predictability:** More stable performance due to reduced reliance on JVM GC.

**How it Connects:** Catalyst selects the physical operators, and Tungsten provides highly optimized implementations for many of these operators, often using whole-stage code generation and unsafe memory operations. You'll see evidence of Tungsten in the physical plan generated by `explain()` (e.g., operators often prefixed or suffixed with terms hinting at code generation or specific optimized algorithms like `SortAggregate` vs `HashAggregate`).

### Putting It All Together: The Execution Flow

1.  You write PySpark code using DataFrame API or SQL.
2.  Spark Driver uses **Catalyst** to parse, analyze, optimize (logical & physical), and generate a physical plan (potentially using cost-based optimization).
3.  When an Action is triggered, the **DAG Scheduler** takes the final physical plan (or RDD lineage).
4.  The DAG Scheduler identifies **shuffle boundaries** (wide dependencies) and breaks the plan into a **DAG of Stages**.
5.  The DAG Scheduler submits ready Stages to the **Task Scheduler**.
6.  The Task Scheduler launches **Tasks** for each partition within a Stage onto available **Executors**.
7.  Executors execute the Tasks, running the optimized bytecode generated by **Catalyst/Tungsten**, leveraging **whole-stage code generation**, **off-heap memory**, and **cache-aware algorithms** for maximum efficiency.
8.  Intermediate results (shuffle data) are written, and final results are sent back to the Driver upon Action completion.

### Practical Implications and Performance Tuning

*   **Monitor the Spark UI:** The Jobs, Stages, and Tasks tabs are invaluable. Look for long-running stages (often shuffle-bound), high task latency variation (skew), task failures, or high GC time (which Tungsten helps mitigate). The SQL tab visualizes the DAG and operator details for DataFrame/SQL queries.
*   **Minimize Shuffles:** Shuffles are expensive (network I/O, disk I/O, serialization). Design transformations to avoid or minimize shuffles where possible. Techniques include:
    *   Using broadcast joins for small tables.
    *   Choosing appropriate partitioning strategies (`repartition`, `partitionBy` on write).
    *   Using window functions instead of group-by where applicable.
*   **Prefer DataFrame/SQL API:** Leverage Catalyst and Tungsten by using the higher-level APIs over RDDs for structured data whenever possible.
*   **Enable AQE (Adaptive Query Execution):** Spark 3.x introduced AQE, which can further optimize queries at runtime based on actual data statistics observed during execution (e.g., dynamically switching join strategies, coalescing shuffle partitions).
*   **Tune Configuration:** Adjust parameters like `spark.sql.shuffle.partitions`, executor memory, cores, and memory configurations (`spark.memory.fraction`, `spark.memory.storageFraction`) based on workload characteristics and cluster resources. Understanding the execution model helps interpret the effects of these settings.

### Conclusion

Spark's execution model, powered by the DAG Scheduler, Catalyst Optimizer, and Tungsten engine, is a sophisticated system designed for high-performance distributed data processing. Understanding how Jobs, Stages, and Tasks form the execution hierarchy, how Catalyst optimizes queries, and how Tungsten accelerates low-level execution empowers developers to write more efficient code, diagnose bottlenecks effectively, and tune Spark applications for optimal performance on modern hardware. This knowledge transforms Spark from a black box into a transparent and controllable processing engine.

---
**End of Lesson 11 Notes**